## Preprocessing

In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf
import os
from tensorflow.keras.callbacks import ModelCheckpoint

#  Import and read the charity_data.csv.
import pandas as pd 
application_df = pd.read_csv("Resources/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [2]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df = application_df.drop(columns = ['EIN', 'NAME'])
application_df 

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1
...,...,...,...,...,...,...,...,...,...,...
34294,T4,Independent,C1000,ProductDev,Association,1,0,N,5000,0
34295,T4,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
34296,T3,CompanySponsored,C2000,Preservation,Association,1,0,N,5000,0
34297,T5,Independent,C3000,ProductDev,Association,1,0,N,5000,1


In [3]:
# Determine the number of unique values in each column.
for x in application_df.columns:
    print(x, len(application_df[x].unique()))

APPLICATION_TYPE 17
AFFILIATION 6
CLASSIFICATION 71
USE_CASE 5
ORGANIZATION 4
STATUS 2
INCOME_AMT 9
SPECIAL_CONSIDERATIONS 2
ASK_AMT 8747
IS_SUCCESSFUL 2


In [4]:
# Look at APPLICATION_TYPE value counts for binning
counts_app = application_df['APPLICATION_TYPE'].value_counts()
counts_app

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T15        2
T29        2
T17        1
Name: APPLICATION_TYPE, dtype: int64

In [5]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
application_types_to_replace = list(counts_app[counts_app<500].index)
application_types_to_replace

# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
application_df['APPLICATION_TYPE'].value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: APPLICATION_TYPE, dtype: int64

In [6]:
# Look at CLASSIFICATION value counts for binning
counts_binning = application_df['CLASSIFICATION'].value_counts()

In [7]:
# You may find it helpful to look at CLASSIFICATION value counts >1
counts_classification = counts_binning[counts_binning>1]
counts_classification

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
C1230       36
C1400       34
C2300       32
C7200       32
C1240       30
C8000       20
C7120       18
C1500       16
C6000       15
C1800       15
C1250       14
C8200       11
C1238       10
C1278       10
C1237        9
C1235        9
C7210        7
C2400        6
C4100        6
C1720        6
C1257        5
C1600        5
C0           3
C2710        3
C1260        3
C1256        2
C1234        2
C3200        2
C1246        2
C1267        2
Name: CLASSIFICATION, dtype: int64

In [8]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
classifications_to_replace = list (counts_binning[counts_binning<1880].index)
classifications_to_replace

# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")
    
# Check to make sure binning was successful
application_df['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: CLASSIFICATION, dtype: int64

In [9]:
# Convert categorical data to numeric with `pd.get_dummies`
application_df = pd.get_dummies(application_df,dtype=float)
application_df.head()

,STATUS,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1,5000,1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,1,108590,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,1,5000,0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,1,6692,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,1,142590,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [10]:
# Split our preprocessed data into our features and target arrays
y = application_df['IS_SUCCESSFUL'].values
y

# Take out IS_SUCCESSFUL
X = application_df.drop('IS_SUCCESSFUL', axis=1).values
X

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X,y,random_state = 42)

In [11]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Optimize the Model Number 1

In [18]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
input_layer = len(X_train_scaled[0])
hidden_nodes_L1= 100
hidden_nodes_L2 = 50

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_L1, activation="relu", input_dim=input_layer))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_L2, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_10 (Dense)            (None, 100)               4400      
                                                                 
 dense_11 (Dense)            (None, 50)                5050      
                                                                 
 dense_12 (Dense)            (None, 1)                 51        
                                                                 
Total params: 9,501
Trainable params: 9,501
Non-trainable params: 0
_________________________________________________________________


In [19]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [20]:
# Train the model
fit_model = nn.fit(X_train_scaled, y_train, epochs=100)

Epoch 1/100
804/804 [==============================] - 3s 3ms/step - loss: 0.5691 - accuracy: 0.7205
Epoch 2/100
804/804 [==============================] - 3s 3ms/step - loss: 0.5562 - accuracy: 0.7278
Epoch 3/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5524 - accuracy: 0.7299
Epoch 4/100
804/804 [==============================] - 3s 3ms/step - loss: 0.5504 - accuracy: 0.7294
Epoch 5/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5500 - accuracy: 0.7301
Epoch 6/100
804/804 [==============================] - 3s 4ms/step - loss: 0.5491 - accuracy: 0.7310
Epoch 7/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5476 - accuracy: 0.7322
Epoch 8/100
804/804 [==============================] - 3s 4ms/step - loss: 0.5471 - accuracy: 0.7321
Epoch 9/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5462 - accuracy: 0.7319
Epoch 10/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5454 - accura

804/804 [==============================] - 1s 2ms/step - loss: 0.5341 - accuracy: 0.7404
Epoch 82/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5339 - accuracy: 0.7397
Epoch 83/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5340 - accuracy: 0.7400
Epoch 84/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5340 - accuracy: 0.7409
Epoch 85/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5337 - accuracy: 0.7403
Epoch 86/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5338 - accuracy: 0.7401
Epoch 87/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5339 - accuracy: 0.7408
Epoch 88/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5338 - accuracy: 0.7402
Epoch 89/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5334 - accuracy: 0.7404
Epoch 90/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5336 - accuracy: 

In [22]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5716 - accuracy: 0.7293 - 312ms/epoch - 1ms/step
Loss: 0.5716118216514587, Accuracy: 0.7293294668197632


## Optimize the Model Number 2

In [23]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
input_layer = len(X_train_scaled[0])
hidden_nodes_L1= 100
hidden_nodes_L2 = 50
hidden_nodes_L3 = 10

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_L1, activation="sigmoid", input_dim=input_layer))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_L2, activation="sigmoid"))

# Third hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_L3, activation="sigmoid"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_13 (Dense)            (None, 100)               4400      
                                                                 
 dense_14 (Dense)            (None, 50)                5050      
                                                                 
 dense_15 (Dense)            (None, 10)                510       
                                                                 
 dense_16 (Dense)            (None, 1)                 11        
                                                                 
Total params: 9,971
Trainable params: 9,971
Non-trainable params: 0
_________________________________________________________________


In [24]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [25]:
# Train the model
fit_model = nn.fit(X_train_scaled, y_train, epochs=100)

Epoch 1/100
804/804 [==============================] - 3s 3ms/step - loss: 0.6007 - accuracy: 0.6991
Epoch 2/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5745 - accuracy: 0.7264
Epoch 3/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5659 - accuracy: 0.7268
Epoch 4/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5605 - accuracy: 0.7270
Epoch 5/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5576 - accuracy: 0.7281
Epoch 6/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5561 - accuracy: 0.7293
Epoch 7/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5554 - accuracy: 0.7301
Epoch 8/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5551 - accuracy: 0.7303
Epoch 9/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5533 - accuracy: 0.7302
Epoch 10/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5530 - accura

804/804 [==============================] - 1s 2ms/step - loss: 0.5359 - accuracy: 0.7377
Epoch 82/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5361 - accuracy: 0.7390
Epoch 83/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5357 - accuracy: 0.7386
Epoch 84/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5359 - accuracy: 0.7377
Epoch 85/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5354 - accuracy: 0.7385
Epoch 86/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5354 - accuracy: 0.7379
Epoch 87/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5355 - accuracy: 0.7397
Epoch 88/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5354 - accuracy: 0.7383
Epoch 89/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5352 - accuracy: 0.7386
Epoch 90/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5350 - accuracy: 

In [26]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5590 - accuracy: 0.7266 - 438ms/epoch - 2ms/step
Loss: 0.5590028166770935, Accuracy: 0.7266472578048706


In [27]:
# Export our model to HDF5 file
nn.save("AlphabetSoupCharity_Optimisation.h5")

## Optimize the Model Number 3

In [28]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
input_layer = len(X_train_scaled[0])
hidden_nodes_L1= 100
hidden_nodes_L2 = 50
hidden_nodes_L3 = 10

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_L1, activation="sigmoid", input_dim=input_layer))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_L2, activation="sigmoid"))

# Third hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_L3, activation="sigmoid"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_17 (Dense)            (None, 100)               4400      
                                                                 
 dense_18 (Dense)            (None, 50)                5050      
                                                                 
 dense_19 (Dense)            (None, 10)                510       
                                                                 
 dense_20 (Dense)            (None, 1)                 11        
                                                                 
Total params: 9,971
Trainable params: 9,971
Non-trainable params: 0
_________________________________________________________________


In [29]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [30]:
# Train the model
fit_model = nn.fit(X_train_scaled, y_train, epochs=200)

Epoch 1/200
804/804 [==============================] - 2s 2ms/step - loss: 0.5994 - accuracy: 0.6968
Epoch 2/200
804/804 [==============================] - 1s 2ms/step - loss: 0.5722 - accuracy: 0.7217
Epoch 3/200
804/804 [==============================] - 1s 2ms/step - loss: 0.5652 - accuracy: 0.7261
Epoch 4/200
804/804 [==============================] - 1s 2ms/step - loss: 0.5604 - accuracy: 0.7271
Epoch 5/200
804/804 [==============================] - 1s 2ms/step - loss: 0.5582 - accuracy: 0.7277
Epoch 6/200
804/804 [==============================] - 1s 2ms/step - loss: 0.5560 - accuracy: 0.7296
Epoch 7/200
804/804 [==============================] - 2s 3ms/step - loss: 0.5551 - accuracy: 0.7304
Epoch 8/200
804/804 [==============================] - 1s 2ms/step - loss: 0.5539 - accuracy: 0.7297
Epoch 9/200
804/804 [==============================] - 1s 2ms/step - loss: 0.5539 - accuracy: 0.7290
Epoch 10/200
804/804 [==============================] - 1s 2ms/step - loss: 0.5528 - accura

804/804 [==============================] - 2s 3ms/step - loss: 0.5359 - accuracy: 0.7383
Epoch 82/200
804/804 [==============================] - 1s 2ms/step - loss: 0.5358 - accuracy: 0.7386
Epoch 83/200
804/804 [==============================] - 1s 2ms/step - loss: 0.5359 - accuracy: 0.7384
Epoch 84/200
804/804 [==============================] - 2s 2ms/step - loss: 0.5355 - accuracy: 0.7389
Epoch 85/200
804/804 [==============================] - 2s 2ms/step - loss: 0.5354 - accuracy: 0.7389
Epoch 86/200
804/804 [==============================] - 1s 2ms/step - loss: 0.5354 - accuracy: 0.7385
Epoch 87/200
804/804 [==============================] - 2s 2ms/step - loss: 0.5353 - accuracy: 0.7384
Epoch 88/200
804/804 [==============================] - 1s 2ms/step - loss: 0.5353 - accuracy: 0.7392
Epoch 89/200
804/804 [==============================] - 2s 2ms/step - loss: 0.5352 - accuracy: 0.7395
Epoch 90/200
804/804 [==============================] - 1s 2ms/step - loss: 0.5350 - accuracy: 

804/804 [==============================] - 2s 2ms/step - loss: 0.5310 - accuracy: 0.7414
Epoch 161/200
804/804 [==============================] - 1s 2ms/step - loss: 0.5307 - accuracy: 0.7407
Epoch 162/200
804/804 [==============================] - 1s 2ms/step - loss: 0.5307 - accuracy: 0.7416
Epoch 163/200
804/804 [==============================] - 1s 2ms/step - loss: 0.5307 - accuracy: 0.7410
Epoch 164/200
804/804 [==============================] - 2s 3ms/step - loss: 0.5307 - accuracy: 0.7411
Epoch 165/200
804/804 [==============================] - 3s 3ms/step - loss: 0.5305 - accuracy: 0.7409
Epoch 166/200
804/804 [==============================] - 3s 3ms/step - loss: 0.5305 - accuracy: 0.7407
Epoch 167/200
804/804 [==============================] - 3s 3ms/step - loss: 0.5306 - accuracy: 0.7411
Epoch 168/200
804/804 [==============================] - 2s 3ms/step - loss: 0.5307 - accuracy: 0.7414
Epoch 169/200
804/804 [==============================] - 1s 2ms/step - loss: 0.5305 - a

In [31]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 1s - loss: 0.5713 - accuracy: 0.7287 - 630ms/epoch - 2ms/step
Loss: 0.5713051557540894, Accuracy: 0.7287463545799255
